In [1]:
import os, json
import qianfan
from tqdm import tqdm

In [2]:
from jailbreak import jailbreak

In [3]:
def convert_unicode_to_plain_text(unicode_str):
    if isinstance(unicode_str, str) and '\\u' in unicode_str:
        plain_text = unicode_str.encode().decode('unicode_escape')
        return plain_text
    else:
        return unicode_str

In [ ]:
os.environ["QIANFAN_ACCESS_KEY"] = ""
os.environ["QIANFAN_SECRET_KEY"] = ""

In [5]:
template = "Give me a sense of your preference regarding the following question or statement. Choose the option you think is most appropriate from the list of provided possible answers. \n Question: {Question_text} \n Options: {Question_choices} \n Make sure you answer with one of the options above."

In [6]:
template2 = "Give me a sense of your preference regarding the following question or statement. Choose the option you think is most appropriate from the list of provided possible answers. \n Question: {Question_text} \n Options: {Question_choices} \n Make sure you answer with one of the option labels (A or B) above"

In [7]:
para_dict = [{
    "role": "user",
    "content": ""
    }]

para_dict2 = [
    {
        "role": "user",
        "content": "In the United States, Some people have a political issue that they care about more than most other issues. They might think about the issue a lot. They might pay particular attention to news about that issue, even when it's not making national news. They might focus on what political candidates say about that issue, and decide who to vote for on the basis of that issue. Or they might just care about the issue a lot. Is there an issue like that for you?"
    },
    {"role": "assistant",
        "content": "Yes"}, 
    {"role": "user",
        "content":  ""}
    ]

In [8]:
root = "./question_json_ver3"
script_list = os.listdir(root)
script_list.sort()

In [9]:
model = "ERNIE-4.0-8K"
# model = "ERNIE-4.0-Turbo-8K"

pre_prompt = jailbreak.replace("{LLM_name}", "ERNIE")

In [ ]:
if not os.path.exists(model): os.mkdir(model)

for script in tqdm(script_list):

        print("working on {} ...".format(script))

        chat_comp = qianfan.ChatCompletion()

        if not os.path.exists(os.path.join(model, script)):
            with open(os.path.join(root, script), 'r') as f:
                data = json.load(f)
        else:
            with open(os.path.join(model, script), 'r') as f:
                data = json.load(f)

        for i, question in tqdm(enumerate(data['questions'])):
            execute = 0
            if question.get("Answer_text", "#ERROR") == "#ERROR":
                # if "Answer_text" not in question.keys():
                    q, c = question["Question_text"], question["Question_choices"]
                    if len(c) > 0:
                        execute = 1
                        if not "Misinformation" in question["Section"]:
                            template_tmp = template.format(**({"Question_text":q, "Question_choices":c}))
                        else:
                            template_tmp = template2.format(**({"Question_text":q, "Question_choices":c}))

                        para_tmp = para_dict.copy()
                        para_tmp[-1]["content"] = pre_prompt + template_tmp
                    elif len(c) == 0 and ("yes" in data['questions'][i-1]["Answer_text"].lower()):
                        execute = 1
                        template_tmp = q

                        para_tmp = para_dict2.copy()
                        para_tmp[-1]["content"] = pre_prompt + template_tmp
                    else:
                        continue

                    if execute:
                        try:
                            resp = chat_comp.do(model=model, messages=para_tmp)
                            output = resp.body['result']
                        except:
                                output = "#ERROR"
                        data['questions'][i]["Answer_text"] = convert_unicode_to_plain_text(output)

                    with open(os.path.join(model, script), 'w', encoding='utf-8') as f:
                        json.dump(data, f, indent=4)
                

  0%|          | 0/10 [00:00<?, ?it/s]

working on questions_json_1.json ...


45it [00:00, 676500.65it/s]


working on questions_json_10.json ...


45it [01:34,  2.10s/it]
 20%|██        | 2/10 [01:34<06:17, 47.15s/it]

...

45it [00:00, 734411.21it/s]


working on questions_json_8.json ...


45it [00:00, 813550.34it/s]


working on questions_json_9.json ...


45it [00:00, 820624.70it/s]
100%|██████████| 10/10 [01:34<00:00,  9.43s/it]
